In [3]:
import gzip
import numpy as np
import os
import pickle
import random
import scipy.io
import stat
import subprocess
import sys

In [4]:
filename = '/Users/franciscojavierarceo/GitHub/ECBME6040/e6040_hw4_fja2114/data/atis.fold3.pkl.gz'
f = gzip.open(filename, 'rb')
train_set, valid_set, test_set, dic = pickle.load(f)

In [5]:
idx2label = dict((k, v) for v, k in dic['labels2idx'].items())
idx2word = dict((k, v) for v, k in dic['words2idx'].items())

# unpack dataset
train_lex, train_ne, train_y = train_set
valid_lex, valid_ne, valid_y = valid_set
test_lex, test_ne, test_y = test_set

vocsize = len(dic['words2idx'])
nclasses = len(dic['labels2idx'])
nsequences = len(train_lex)

In [9]:
sumy = 0
sumn = 0
for i, (x, y) in enumerate(zip(train_lex, train_y)):
    if len(x)==len(y):
        sumy+=1
    else:
        sumn+=1

In [10]:
sumy, sumn

(3983, 0)

In [27]:
vocsize, nclasses, nsequences

(572, 127, 3983)

In [28]:
[(w, idx2word[w]) for w in train_lex[0]]

[(554, 'what'),
 (23, 'aircraft'),
 (241, 'is'),
 (534, 'used'),
 (358, 'on'),
 (136, 'delta'),
 (193, 'flight'),
 (11, 'DIGITDIGITDIGITDIGIT'),
 (208, 'from'),
 (251, 'kansas'),
 (104, 'city'),
 (502, 'to'),
 (413, 'salt'),
 (256, 'lake'),
 (104, 'city')]

In [29]:
# verifying this
for j in range(len(train_ne)):
    [(w, idx2word[w]) for w in train_ne[j]]

In [30]:
[(w, idx2word[w]) for w in train_ne[0]]

[(0, "'d"),
 (0, "'d"),
 (0, "'d"),
 (0, "'d"),
 (0, "'d"),
 (6, '72s'),
 (0, "'d"),
 (52, 'arizona'),
 (0, "'d"),
 (18, 'actually'),
 (109, 'co'),
 (0, "'d"),
 (18, 'actually'),
 (109, 'co'),
 (109, 'co')]

In [31]:
[(w, idx2label[w]) for w in train_y[0]]

[(126, 'O'),
 (126, 'O'),
 (126, 'O'),
 (126, 'O'),
 (126, 'O'),
 (2, 'B-airline_name'),
 (126, 'O'),
 (43, 'B-flight_number'),
 (126, 'O'),
 (48, 'B-fromloc.city_name'),
 (109, 'I-fromloc.city_name'),
 (126, 'O'),
 (78, 'B-toloc.city_name'),
 (123, 'I-toloc.city_name'),
 (123, 'I-toloc.city_name')]

In [32]:
idx2label

{0: 'B-aircraft_code',
 1: 'B-airline_code',
 2: 'B-airline_name',
 3: 'B-airport_code',
 4: 'B-airport_name',
 5: 'B-arrive_date.date_relative',
 6: 'B-arrive_date.day_name',
 7: 'B-arrive_date.day_number',
 8: 'B-arrive_date.month_name',
 9: 'B-arrive_date.today_relative',
 10: 'B-arrive_time.end_time',
 11: 'B-arrive_time.period_mod',
 12: 'B-arrive_time.period_of_day',
 13: 'B-arrive_time.start_time',
 14: 'B-arrive_time.time',
 15: 'B-arrive_time.time_relative',
 16: 'B-booking_class',
 17: 'B-city_name',
 18: 'B-class_type',
 19: 'B-compartment',
 20: 'B-connect',
 21: 'B-cost_relative',
 22: 'B-day_name',
 23: 'B-day_number',
 24: 'B-days_code',
 25: 'B-depart_date.date_relative',
 26: 'B-depart_date.day_name',
 27: 'B-depart_date.day_number',
 28: 'B-depart_date.month_name',
 29: 'B-depart_date.today_relative',
 30: 'B-depart_date.year',
 31: 'B-depart_time.end_time',
 32: 'B-depart_time.period_mod',
 33: 'B-depart_time.period_of_day',
 34: 'B-depart_time.start_time',
 35: 'B-d

In [33]:
idx2word

{0: "'d",
 1: "'ll",
 2: "'m",
 3: "'re",
 4: "'s",
 5: "'t",
 6: '72s',
 7: '<UNK>',
 8: 'DIGIT',
 9: 'DIGITDIGIT',
 10: 'DIGITDIGITDIGIT',
 11: 'DIGITDIGITDIGITDIGIT',
 12: 'DIGITDIGITDIGITDIGITDIGITDIGIT',
 13: 'a',
 14: 'abbreviation',
 15: 'abbreviations',
 16: 'about',
 17: 'ac',
 18: 'actually',
 19: 'after',
 20: 'afternoon',
 21: 'again',
 22: 'air',
 23: 'aircraft',
 24: 'airfare',
 25: 'airline',
 26: 'airlines',
 27: 'airplane',
 28: 'airplanes',
 29: 'airport',
 30: 'airports',
 31: 'alaska',
 32: 'all',
 33: 'along',
 34: 'also',
 35: 'am',
 36: 'america',
 37: 'american',
 38: 'amount',
 39: 'an',
 40: 'and',
 41: 'angeles',
 42: 'another',
 43: 'any',
 44: 'anywhere',
 45: 'ap',
 46: 'ap57',
 47: 'ap80',
 48: 'approximately',
 49: 'april',
 50: 'are',
 51: 'area',
 52: 'arizona',
 53: 'around',
 54: 'arrange',
 55: 'arrangements',
 56: 'arrival',
 57: 'arrivals',
 58: 'arrive',
 59: 'arrives',
 60: 'arriving',
 61: 'as',
 62: 'at',
 63: 'atl',
 64: 'atlanta',
 65: 'augu

In [1]:
# Experiment 1 -- a simple rnn
import hw4a as h
h.test_rnnslu(nhidden=200, win=5, emb_dimension=50, normal=True, nepochs=5)

Using gpu device 0: GeForce GT 750M (CNMeM is disabled, cuDNN 5004)
/Users/franciscojavierarceo/anaconda2/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


Couldn't import dot_parser, loading of dot files will not be possible.
verbose: True
normal: True
win: 5
savemodel: False
fold: 3
seed: 345
emb_dimension: 50
nepochs: 5
nhidden: 200
decay: True
lr: 0.0970806646813
folder: ../result
... loading the dataset
... building the model
... training


NameError: global name 'urllib' is not defined

# RNN-LSTM in Keras

In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
from keras.datasets import imdb
from keras.optimizers import RMSprop
from sklearn.metrics import roc_auc_score

Using Theano backend.
Using gpu device 0: GeForce GT 750M (CNMeM is disabled, cuDNN 5004)

Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
max_features = 200
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

In [3]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features)

In [4]:
# Oirginal dictionary has the numeric index as values and the keys as the words --- not helpful for revsering
windx = imdb.get_word_index()
# Fastest way to handle this is to reverse the dictionary to get the numeric values to be the dictionary keys 
rwindx = {v:k for k,v in windx.iteritems()}

In [5]:
# Proof it works
print( windx['fawn'])
print( rwindx[34701])

34701
fawn


In [6]:
# What this looks like for a single row
[rwindx[x] for x in X_train[0]]

['the',
 'as',
 'you',
 'with',
 'out',
 'and',
 'and',
 'and',
 'and',
 'their',
 'and',
 'and',
 'had',
 'and',
 'of',
 'lot',
 'from',
 'and',
 'to',
 'have',
 'after',
 'out',
 'and',
 'never',
 'more',
 'and',
 'and',
 'it',
 'so',
 'and',
 'and',
 'to',
 'years',
 'of',
 'every',
 'never',
 'going',
 'and',
 'and',
 'and',
 'or',
 'of',
 'every',
 'and',
 'and',
 'movie',
 'and',
 'her',
 'was',
 'and',
 'of',
 'enough',
 'more',
 'with',
 'is',
 'now',
 'and',
 'film',
 'as',
 'you',
 'of',
 'and',
 'and',
 'and',
 'of',
 'you',
 'than',
 'him',
 'that',
 'with',
 'out',
 'and',
 'her',
 'get',
 'for',
 'was',
 'and',
 'of',
 'you',
 'movie',
 'and',
 'movie',
 'that',
 'with',
 'and',
 'but',
 'and',
 'to',
 'story',
 'and',
 'that',
 'in',
 'and',
 'in',
 'character',
 'to',
 'of',
 'and',
 'and',
 'with',
 'and',
 'had',
 'and',
 'they',
 'of',
 'here',
 'that',
 'with',
 'her',
 'and',
 'to',
 'have',
 'does',
 'when',
 'from',
 'why',
 'what',
 'have',
 'and',
 'they',
 'is

In [7]:
# What this looks like for a single row
[rwindx[x] for x in X_test[0]]

['the',
 "don't",
 'be',
 'and',
 'and',
 'movie',
 'give',
 'say',
 'to',
 'and',
 'with',
 'and',
 'his',
 'in',
 'and',
 'of',
 'and',
 'br',
 'of',
 'you',
 'and',
 'and',
 'with',
 'and',
 'movie',
 'and',
 'br',
 'and',
 'and',
 'it',
 'of',
 'and',
 'in',
 'as',
 'and',
 'was',
 'and',
 'her',
 'film',
 'be',
 'and',
 'was',
 'after',
 'and',
 'only',
 'and',
 'and',
 'of',
 'br',
 'of',
 'on',
 'these',
 'and',
 'we',
 'in',
 'as',
 'and',
 'be',
 'and',
 'br',
 'and',
 'with',
 'and',
 'and',
 'movie',
 'all',
 'real',
 'one',
 'will',
 'and',
 'but',
 'that']

In [13]:
emb_dim = 50
# RNN for the regular embedings
print('Loading data...')

# (X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features)

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train_rnn = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test_rnn = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train_rnn.shape)
print('X_test shape:', X_test_rnn.shape)

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, emb_dim, dropout=0.2))
model.add(LSTM(emb_dim, input_shape=(maxlen, len(rwindx))))
model.add(Dense(len(rwindx)))
model.add(Activation('softmax'))


# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
X_train shape: (25000, 80)
X_test shape: (25000, 80)
Build model...


In [14]:
print('Train...')
model.fit(X_train_rnn, batch_size=batch_size, nb_epoch=1,
          validation_data=(X_test_rnn), verbose=0)

Train...


TypeError: fit() takes at least 3 arguments (6 given)

In [ ]:
out = model.evaluate(X_test, batch_size=batch_size)

## This is the original RNN-MLP for IMDB Classification

In [ ]:
# This is the original IMDB model, it's an LSTM feeding into an MLP

print('Loading data...')

(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features)

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128, dropout=0.2))
model.add(LSTM(128, dropout_W=0.2, dropout_U=0.2))  # try using a GRU instead, for fun
model.add(Dense(1))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=1,
          validation_data=(X_test, y_test), verbose=0)
score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)
ypred_tst = model.predict(X_test)
# Pretty good AUC
print('Test AUC:', roc_auc_score(y_test,ypred_tst))

## Nietzsche Text Generation

In [16]:
from keras.utils.data_utils import get_file

path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

corpus length: 600901
total chars: 59
nb sequences: 200287
Vectorization...


In [21]:
X.shape

(200287, 40, 59)

In [23]:
X[0]

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ..., 
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]], dtype=bool)

In [17]:
y[0]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False], dtype=bool)

In [ ]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y, batch_size=128, nb_epoch=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()